# Handing off a running notebook between frontends

In [3]:
for i in range(30):
    print(i, end=' ')
    import time; time.sleep(1)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

In [11]:
i

29

## Who updates the outputs under a cell?

[Kernels zmq protocol](https://jupyter-protocol.readthedocs.io/) only understands [execute->results](https://stackoverflow.com/a/79339022/239657), _not even aware notebooks exist_!

### historically

Originally server was dumb, necessary for browser to access kernel and file system but all smarts was in frontend JS.  
The execution outputs and results were followed by frontend via `/api/kernels/{ID}/channels` websocket transporting zmq messages pretty much 1:1.  

It was frontend writing .ipynb file (last-write-wins) via another HTTP API `/api/contents/{PATH}`:

```mermaid
graph LR
frontend@{ shape: process } 
server@{ shape: rounded } 
kernel@{ shape: hex }

subgraph jupyter-server
    direction TB
    style jupyter-server rx:2ex,ry:2ex
    req_ex(["channels websocket"])
    req_edit(["PUT contents"])
end

file[".ipynb file"]@{ shape: card }

frontend -->|execute_request on 'shell' channel| req_ex <-->|zmq| kernel
req_ex ==>|results on 'iopub'... + execute_reply on 'shell'| frontend ==>|updated ipynb JSON| req_edit --> file
```

### move to Yjs, by 2022

Gradually [jupyter-collaboration](https://jupyterlab-realtime-collaboration.readthedocs.io/) extension introduced collaboration via shared [Yjs CRDT](https://docs.yjs.dev/)...

In 2022, the last major piece fell into place: saving .ipynb files switched from separate contents API request to Yjs websocket: https://github.com/jupyterlab/jupyterlab/pull/12360  
but to this day is still by default _driven by frontend_:

```mermaid
graph LR
frontends["frontends, each has Y model"]@{ shape: processes } 
kernel@{ shape: hex }

subgraph jupyter-server
    direction TB
    style jupyter-server rx:2ex,ry:2ex
    req_ex(["channels websocket"])
    req_edit(["Yjs websocket"])
    Ymodel["Y model"]@{ shape: db }
end
    
direction TB
file[.ipynb file]@{ shape: card }

frontends --->|execute_request on 'shell' channel| req_ex <-->|zmq| kernel
req_ex ==>|results on 'iopub'... + execute_reply on 'shell'| frontends ==>|Y edit outputs| req_edit <==> Ymodel --> file
```

* contents API still used for e.g. saving checkpoints?  And file browsing, etc.
* I'm drawing the Y model as in-memory, but it's also [backed by hidden `.jupyter_ystore.db` file](https://jupyterlab-realtime-collaboration.readthedocs.io/en/latest/configuration.html).

## Closing tab + reopening
## (AKA if a cell executes in a forest with no frontends to hear it...)

* In absence of frontend, There is logic to buffer outputs which _maybe_ works when you temporary lose network or reopen _same_ tab.   

* Mostly, buffered outputs get lost in practice 😢 https://github.com/jupyter-server/jupyter_server/issues/1274, esp. if you connect from different browser.
  * work coord nation repo https://github.com/jupyter-server/message_replay — abandoned

### 2024+ opt-in Server-side execution! 🎉
Fix by cleaner architecture!  Frontend just fires off new simpler `api/kernels/{ID}/execute` and server is [responsible](https://github.com/datalayer/jupyter-server-nbmodel/blob/c44d3684c91d95fc5e59514aea8ffd44c67c2878/jupyter_server_nbmodel/handlers.py#L264-L267) for following outputs + results messages and _updating Y model_.  
Any frontends then observe the new outputs via Yjs sync (even if they were offline/came later).

```mermaid
graph LR
frontends["frontends, each has Yjs model"]@{ shape: processes } 
server@{ shape: rounded } 
kernel@{ shape: hex }

subgraph server
    direction TB
    style server rx:2ex,ry:2ex
    req_ex(["POST execute"])
    req_edit(["Yjs websocket"])
    Ymodel["Y model"]@{ shape: db }
end

direction TB
file[.ipynb file]@{ shape: card }

frontends -->|HTTP| req_ex --->|zmq execute| kernel
kernel --->|zmq results...| req_ex
req_ex ==>|Y edit outputs| Ymodel
req_edit <==> Ymodel --> file
frontends <==>|Y sync| req_edit
```
  
collaboration model landed in https://github.com/jupyterlab/jupyter-collaboration/pull/279  
Two options to use:

* jupyter-server with extension https://github.com/datalayer/jupyter-server-nbmodel/
  ```
  uvx --with "jupyter_server_nbmodel[lab]" jupyter lab
  ```

* jupyverse with opt-in, install per https://github.com/jupyterlab/jupyter-collaboration/pull/279 ?
  ```
  jupyverse --set kernels.require_yjs=true --set jupyterlab.server_side_execution
  ```
  harder to find compatible versions?

Still less tested, may have rough edges...  Widgets kinda work?

* [jpterm](https://codeberg.org/davidbrochart/jpterm) needs `--collaborative --experimental` mode to use it.  It also really needs jupyter-server to [disable ALL auth](https://davidbrochart.github.io/jpterm/usage/CLI/) which is bad idea!
  See also https://codeberg.org/davidbrochart/jpterm/pulls/3

----

### P.S. the overall trajectory of relying more and more on CRDT model is classic [Local First](https://localfirstweb.dev/) success story 🌟

### Alternative: keep jpterm running!
If you want to start notebook "headless" and connect later, start `jpterm --server '...?token=...'` inside tmux, and just leave it running.  
Y collaboration means no problem using a browser notebook in parallel.  
This way, you *don't need server-side execution*, it'll work with jpterm relaying outputs.  Later executions (whether client- or server- side) can be triggered by notebook.